In [2]:
from langchain.document_loaders import WebBaseLoader

#add website data
URL = ["https://www.apple.com/apple-vision-pro/",
	"https://www.theverge.com/24054862/apple-vision-pro-review-vr-ar-headset-features-price"]

#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()


ModuleNotFoundError: No module named 'langchain'

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)


In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
	api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)


In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking,embeddings)


In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "Tell me the pros and cons of having an Apple Vision Pro?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)


In [ ]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
					model_kwargs={"temperature":0.5,
									"max_new_tokens":512,
									"max_length":64
									})

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])
